# Spark Group Assignment
### Challenge 1 | Network Intrusion Detection

Group O-2-8

Agenda
1. Spark-Setup / Load Data
2. Inspect Data
3. Preprocess Data
4. Create A Model
5. Make Predictions
6. Evaluate Predictions

## 1. Spark Setup

In [8]:
import os
print(os.environ['SPARK_HOME'])
dataset_path="/home/ubuntu/challenge_1/"

/usr/local/software/spark


In [9]:
import pandas as pd

In [10]:
#import findspark
#findspark.init()
import pyspark

In [11]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .master("local") \
    .appName("Dataset") \
    .getOrCreate()

In [12]:
spark.version

'2.2.0'

### 1.1 Data Loading

Data inspection shows that the data does not have a header. Therefore we are going to use a simple for loop to assign the correct labelling to the columns. Furthermore, we are assignung the variable "connection" to the different types of network intrusion attacks. The connection types fall into the following categories:

* DOS: denial-of-service, e.g. syn flood;
* R2L: unauthorized access from a remote machine, e.g. guessing password;
* U2R:  unauthorized access to local superuser (root) privileges, e.g., various buffer overflow attacks;
* probing: surveillance and other probing, e.g., port scanning.
* normal: no attack was identified

#### 1.1.1 Train Data

In [13]:
df = spark.read \
    .option("inferSchema", "true") \
    .csv("file://"+dataset_path+"full.data")

In [14]:
features=["duration", "protocol_type", "service", "flag", "src_bytes","dst_bytes", \
          "land","wrong_fragment","urgent","hot","num_failed_logins","logged_in", \
          "num_compromised","root_shell","su_attempted","num_root","num_file_creations", \
          "num_shells","num_access_files","num_outbound_cmds","is_host_login","is_guest_login", \
          "count","srv_count","serror_rate","srv_serror_rate","rerror_rate","srv_rerror_rate",\
          "same_srv_rate","diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count", \
          "dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate", \
          "dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate","dst_host_rerror_rate",\
          "dst_host_srv_rerror_rate"]

target=["connection"]

fieldnames=features+target

rawnames=df.schema.names

# Create a small function
def updateColNames(df,oldnames,newnames):
    for i in range(len(newnames)):
        df=df.withColumnRenamed(oldnames[i], newnames[i])
    return df

df=updateColNames(df,rawnames,fieldnames)

# df.printSchema()

#### 1.1.2 Creating new attack variable 'label'

Regarding the scope of this assignment, there is no need to classify attack types into the correct group (i.e probing or DOS). We simply have to identify whether or not an attack is taking place. Thus, we are creating a new boolean column 'lable':

* Assign the value '0' for no attack (=normal)
* Assign the value '1' for attack

In [15]:
# Adding a Boolean column for attack (=1) or normal (=0)
from pyspark.sql.functions import when

df = df.withColumn('label', when(df["connection"] == 'normal.', 0).otherwise(1))

df.groupBy('label').count().show()

+-----+-------+
|label|  count|
+-----+-------+
|    1|3925650|
|    0| 972781|
+-----+-------+



#### 1.2 Loading Test Data

We have to repeat the same process for the test data:

* Assign column names
* Create new column 'label'

In [16]:
df_test = spark.read \
    .option("inferSchema", "true") \
    .csv("file://"+dataset_path+"corrected")

In [17]:
features_test=["duration", "protocol_type", "service", "flag", "src_bytes","dst_bytes", \
          "land","wrong_fragment","urgent","hot","num_failed_logins","logged_in", \
          "num_compromised","root_shell","su_attempted","num_root","num_file_creations", \
          "num_shells","num_access_files","num_outbound_cmds","is_host_login","is_guest_login", \
          "count","srv_count","serror_rate","srv_serror_rate","rerror_rate","srv_rerror_rate",\
          "same_srv_rate","diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count", \
          "dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate", \
          "dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate","dst_host_rerror_rate",\
          "dst_host_srv_rerror_rate"]

target_test=["connection"]

fieldnames_test=features_test+target_test

rawnames_test=df_test.schema.names

# Create a small function
def updateColNames_test(df_test,oldnames,newnames):
    for i in range(len(newnames)):
        df_test=df_test.withColumnRenamed(oldnames[i], newnames[i])
    return df_test

df_test=updateColNames(df_test,rawnames,fieldnames)

# df_test.printSchema()

In [18]:
# Adding a Boolean column for attack (=1) or normal (=0)
from pyspark.sql.functions import when

df_test = df_test.withColumn('label', when(df_test["connection"] == 'normal.', 0).otherwise(1))

df_test.groupBy('label').count().show()

+-----+------+
|label| count|
+-----+------+
|    1|250436|
|    0| 60593|
+-----+------+



## 2. Data Inspection


* How many records do we have?
* What is the schema of our data?
* Is it numerical , is it categorical?
* Visualize your data

In [19]:
# Print the number of records in the data frame
print('Nb. of records  : %d' % df.count())

Nb. of records  : 4898431


In [20]:
df.select('duration','src_bytes','dst_bytes','wrong_fragment','num_failed_logins').describe().show()

+-------+-----------------+------------------+------------------+--------------------+--------------------+
|summary|         duration|         src_bytes|         dst_bytes|      wrong_fragment|   num_failed_logins|
+-------+-----------------+------------------+------------------+--------------------+--------------------+
|  count|          4898431|           4898431|           4898431|             4898431|             4898431|
|   mean|48.34243046395876|1834.6211752293746|1093.6228137132073|6.487791703098401E-4|3.205107921291532E-5|
| stddev|723.3298112546812| 941431.0744911365| 645012.3337425214| 0.04285433675493731|0.007299407575927214|
|    min|                0|                 0|                 0|                   0|                   0|
|    max|            58329|        1379963888|        1309937401|                   3|                   5|
+-------+-----------------+------------------+------------------+--------------------+--------------------+



### 2.1 Exploring numercial variables

In total, there are 28 numercial variables in our dataset:

* XX continous 22)
* XX boolean (6)

We are using agg() operations in order to compare means between attack and non-attack networks and receive a couple of insights:

* Duration: the mean duration of normal connection is longer
* Dst_bytes: the mean number of data bytes from destination to source is 6x greater
* Hot: the mean number of 'hot' indiactors is 15x smaller for attacks

In [21]:
### Compare averages of numcerical features between 

In [22]:
# Some stats on numerical features
df.groupBy('label').agg({'duration': 'mean'}).orderBy("avg(duration)", ascending = False).show(30)

+-----+------------------+
|label|     avg(duration)|
+-----+------------------+
|    0|217.82472416710442|
|    1|6.3445052411702525|
+-----+------------------+



In [23]:
# Some stats on numerical features
df.groupBy('label').agg({'src_bytes': 'mean'}).orderBy("avg(src_bytes)", ascending = False).show(30)

+-----+------------------+
|label|    avg(src_bytes)|
+-----+------------------+
|    1| 1923.030449734439|
|    0|1477.8462500809535|
+-----+------------------+



In [24]:
df.groupBy('label').agg({'dst_bytes': 'mean'}).orderBy("avg(dst_bytes)", ascending = False).show(30)

+-----+------------------+
|label|    avg(dst_bytes)|
+-----+------------------+
|    0|3234.6501113816985|
|    1| 563.0735605568505|
+-----+------------------+



In [25]:
# Some stats on numerical features
df.groupBy('label').agg({'wrong_fragment': 'mean'}).orderBy("avg(wrong_fragment)", ascending = False).show(30)

+-----+--------------------+
|label| avg(wrong_fragment)|
+-----+--------------------+
|    1|8.095474634773859E-4|
|    0|                 0.0|
+-----+--------------------+



In [26]:
# Some stats on numerical features
df.groupBy('label').agg({'hot': 'mean'}).orderBy("avg(hot)", ascending = False).show(30)

+-----+--------------------+
|label|            avg(hot)|
+-----+--------------------+
|    0| 0.04953530136793379|
|    1|0.003244812960910932|
+-----+--------------------+



In [27]:
# Some stats on numerical features
df.groupBy('label').agg({'num_failed_logins': 'mean'}).orderBy("avg(num_failed_logins)", ascending = False).show(30)

+-----+----------------------+
|label|avg(num_failed_logins)|
+-----+----------------------+
|    0|   9.86861379899484E-5|
|    1|  1.553882796479563...|
+-----+----------------------+



### 2.2. Exploring the categorical variables

Again, we are using grouby() commands to explore the categorical variables and their count().

* protocol_type (3 distinct types)
* service       (70 distinct types)
* flag          (11 distinct types)
* connection    (21 distinct types)

in term of the number of categories and count()

In [31]:
# How many distict flags we have
df.groupby('protocol_type').count().show()

+-------------+-------+
|protocol_type|  count|
+-------------+-------+
|          tcp|1870598|
|          udp| 194288|
|         icmp|2833545|
+-------------+-------+



In [32]:
# How many distict services we have
df.groupby('service').count().show()

+---------+-----+
|  service|count|
+---------+-----+
|   telnet| 4277|
|      ftp| 5214|
|     auth| 3382|
| iso_tsap| 1052|
|   systat| 1056|
|     name| 1067|
|  sql_net| 1052|
|    ntp_u| 3833|
|      X11|  135|
|    pop_3| 1981|
|     ldap| 1041|
|  discard| 1059|
|   tftp_u|    3|
|   Z39_50| 1078|
|  daytime| 1056|
| domain_u|57782|
|    login| 1045|
|     smtp|96554|
|http_2784|    1|
|      mtp| 1076|
+---------+-----+
only showing top 20 rows



In [33]:
# How many distict flags we have
df.groupby('flag').count().show()

+------+-------+
|  flag|  count|
+------+-------+
|RSTOS0|    122|
|    S3|     50|
|    SF|3744328|
|    S0| 869829|
|   OTH|     57|
|   REJ| 268874|
|  RSTO|   5344|
|  RSTR|   8094|
|    SH|   1040|
|    S2|    161|
|    S1|    532|
+------+-------+



In [34]:
df.groupby('connection').count()\
    .orderBy('count', ascending =False)\
    .show(100)

+----------------+-------+
|      connection|  count|
+----------------+-------+
|          smurf.|2807886|
|        neptune.|1072017|
|         normal.| 972781|
|          satan.|  15892|
|        ipsweep.|  12481|
|      portsweep.|  10413|
|           nmap.|   2316|
|           back.|   2203|
|    warezclient.|   1020|
|       teardrop.|    979|
|            pod.|    264|
|   guess_passwd.|     53|
|buffer_overflow.|     30|
|           land.|     21|
|    warezmaster.|     20|
|           imap.|     12|
|        rootkit.|     10|
|     loadmodule.|      9|
|      ftp_write.|      8|
|       multihop.|      7|
|            phf.|      4|
|           perl.|      3|
|            spy.|      2|
+----------------+-------+



### 2.3 Exploring data visually (@Adolfo)

tbd

In [35]:
# 3a. Create a in-memory DataFrame 
# df2.registerTempTable("network_data")

## 3. Preprocess Data

The data inspetion shows that our dataset contains three categorical variables:

* protocol_type
* service
* flag

We are going to use StringIndexer, OneHotEncoder, Vector Assembler and a Pipeline to compute feature transformation.

* **StringIndexer**: converts a single column to an index column (similar to a factor column in R)
* **OneHotEncoder**: One-hot encoding maps a column of label indices to a column of binary vectors, with at most a single one-value. This encoding allows algorithms which expect continuous features, such as Logistic Regression, to use categorical features.
* **VectorAssembler**: A transformer that combines a given list of columns into a single vector column.
* **Pipelines**: Facilitates the creation, tuning, and inspection of practical ML workflows. A Spark Pipeline is specified as a sequence of stages, and each stage is either a Transformer or an Estimator. These stages are run in order, and the input DataFrame is transformed as it passes through each stage. 


In [36]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler

#### 3.1. Transformations

In [98]:
from pyspark.ml import Pipeline

categoricalColumns = [ \
           "protocol_type", "service", "flag"]

stages = [] # stages in our Pipeline
for col in categoricalColumns:
  
  # Category Indexing with StringIndexer
  indexer = StringIndexer(inputCol=col, outputCol=col+"_index")
   
  # Use OneHotEncoder to convert categorical variables into binary SparseVectors
  # encoder = OneHotEncoder(inputCol=col+"_index", outputCol=col+"_vector")
  
  # Add stages.  These are not run here, but will run all at once later on.
  stages += [indexer]

#### 3.2 VectorAssembler 

This output will include both the numeric columns and the one-hot encoded binary vector columns in our dataset.

We are not going to use all of the numeric features from the dataset. The most important features have been identified while inspecting the data. 

In [99]:
# Transform all numerical features into a vector using VectorAssembler

numericCols_model = ["duration","src_bytes","dst_bytes","land","wrong_fragment","urgent"]

assemblerInputs = [ col + "_index" for col in categoricalColumns ] + numericCols_model
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
stages += [assembler]

print(assemblerInputs)

['protocol_type_index', 'service_index', 'flag_index', 'duration', 'src_bytes', 'dst_bytes', 'land', 'wrong_fragment', 'urgent']


#### 3.3 Checking the stages

In [100]:
# Check the stages of our pipeline
n=0
for s in stages:
    print('stage number %d %s' %(n,s.getOutputCol()))
    n+=1 

stage number 0 protocol_type_index
stage number 1 service_index
stage number 2 flag_index
stage number 3 features


## 4. Create a Model 
 * Create the model
 * Split data into train and test data
 * Train the model with train data
 * Test model predictions with test data

#### 4.1 Create Pipleline

Group together the stages we defined (feature transformations).

In [101]:
from pyspark.ml import Pipeline
# Create a Pipeline.
pipeline = Pipeline(stages=stages)

transformer = pipeline.fit(df)
transformed_df = transformer.transform(df)

# Focus on the relevant columns and define dataset
selection = ["label", "features"] # + assemblerInputs     # "duration", "src_bytes"
dataset = transformed_df.select(selection)

In [102]:
dataset.limit(5).toPandas()

,label,features
0,0,"(1.0, 2.0, 0.0, 0.0, 215.0, 45076.0, 0.0, 0.0,..."
1,0,"(1.0, 2.0, 0.0, 0.0, 162.0, 4528.0, 0.0, 0.0, ..."
2,0,"(1.0, 2.0, 0.0, 0.0, 236.0, 1228.0, 0.0, 0.0, ..."
3,0,"(1.0, 2.0, 0.0, 0.0, 233.0, 2032.0, 0.0, 0.0, ..."
4,0,"(1.0, 2.0, 0.0, 0.0, 239.0, 486.0, 0.0, 0.0, 0.0)"


#### 4.2 Splitting dataset into train and test

* 70% train | 30% test
* Setting a seed to esnure reproducability of the split

In [103]:
(train_data, test_data) = dataset.randomSplit([0.7, 0.3], seed = 123)
print('Training records : %d' % train_data.count())
print('Test records : %d ' % test_data.count())
train_data.cache()

Training records : 3427798
Test records : 1470633 


DataFrame[label: int, features: vector]

#### 4.3 Create a Logisitc Regression Model

In [104]:
from pyspark.ml.classification import LogisticRegression

# Create initial LogisticRegression model
lr = LogisticRegression(labelCol="label", featuresCol="features", maxIter=10)

# Train model with Training Data
model = lr.fit(train_data)

In [108]:
# Make predictions on test data using the transform() method. Feature have been specified earlier.
predictions = model.transform(test_data)
predictions

DataFrame[label: int, features: vector, rawPrediction: vector, probability: vector, prediction: double]

In [109]:
predictions.limit(5).toPandas()

,label,features,rawPrediction,probability,prediction
0,0,"(1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)","[2.26141401484, -2.26141401484]","[0.905630547549, 0.0943694524514]",0.0
1,0,"(1.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)","[2.31846268861, -2.31846268861]","[0.910394611644, 0.0896053883558]",0.0
2,0,"(1.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)","[2.31846268861, -2.31846268861]","[0.910394611644, 0.0896053883558]",0.0
3,0,"(1.0, 2.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)","[-1.89255377757, 1.89255377757]","[0.130953563468, 0.869046436532]",1.0
4,0,"(1.0, 2.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)","[-1.89255377757, 1.89255377757]","[0.130953563468, 0.869046436532]",1.0


In [107]:
# See model's predictions and probabilities of each prediction class
# You can select any columns in the above schema to view as well. 
# For example's sake we will choose age & occupation
#selected = predictions.select("label", "prediction", "duration","src_bytes")

In [47]:
#selected.limit(5).toPandas()

,label,prediction,duration,src_bytes
0,0,1.0,0,30
1,0,1.0,0,30
2,0,1.0,0,30
3,0,1.0,0,30
4,0,1.0,0,30


#### Evaluation Metrics:

Binary classifiers are used to separate the elements of a given dataset into one of two possible groups (e.g. attack or no attack).

In [110]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Evaluate model
evaluator = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")
score = evaluator.evaluate(predictions)
print('Score is : %03f' % score )

Score is : 0.942774


### Model Selection

** READ THIS : ** : https://spark.apache.org/docs/2.2.1/ml-tuning.html

Model selection consists in using data to find the best model or parameters for a given task.

 * Inspect available parameters for tuning
 * Use CrossValidation or TrainValidationSplit for parameter tuning
 * Both requires the following inputs:
    *  Estimator: algorithm or Pipeline to tune
    *  Set of ParamMaps: parameters to choose from, sometimes called a “parameter grid” to search over
    *  Evaluator: metric to measure how well a fitted Model does on held-out test data

* At a high level, these model selection tools work as follows:

   *  They split the input data into separate training and test datasets.
   * For each (training, test) pair, they iterate through the set of ParamMaps:
   * For each ParamMap, they fit the Estimator using those parameters, get the fitted Model, and evaluate the Model’s performance using the Evaluator.
   
*  They finally select the Model produced by the best-performing set of parameters.

** An interesting blog on parameter tuning ** :https://www.oreilly.com/ideas/big-datas-biggest-secret-hyperparameter-tuning

In [83]:
print(lr.explainParam("regParam"))

regParam: regularization parameter (>= 0). (default: 0.0)


In [84]:
print(lr.explainParam("elasticNetParam"))

elasticNetParam: the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty. (default: 0.0)


#### Create Parameters Grid for Cross Validation
we will create a model for each combination of parameters in the grid specified and evaluate its result

We use :
 3 regularization param values (regParam)
 3 values for maximum nb of iterations
 3 values for elasticNetParam
 The grid will have 3 x 3 x 3 = 27 parameter settings to choose from. 


 Regularization Parameter: 

 (intuitively) is a penalty against complexity. 
 A bigger regParam penalizes "large" weight coefficients ,i.e, 
 tries to avoid our model model picking up "noise," or "deducting a pattern where there is none."
 tries to avoid OVERFITTING

 ElasticNetParam:
 read this : https://en.wikipedia.org/wiki/Elastic_net_regularization

In [ ]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
paramGrid = (ParamGridBuilder()
             .addGrid(lr.regParam, [1, 5, 10])
             .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0])
             .addGrid(lr.maxIter, [1, 5, 10])
             .build())

In [ ]:
# Create 3-fold CrossValidator

# numFolds determines the number of train/test dataset pairs used in the cross-validation
# The cross validation will compute the  average of the evaluation metrics produced by the n models
# by fitting the Estimator on the 3 different (training, test) dataset pairs.

cv = CrossValidator(estimator=lr, estimatorParamMaps=paramGrid, evaluator=evaluator, numFolds=3)

# Run cross validations
cvModel = cv.fit(train_data)
# this may take some of time (depends on the amount of models that we're creating and testing)

In [ ]:
# Use test set here so we can measure the accuracy of our model on new data
predictions = cvModel.transform(test_data)
# cvModel uses the best model found from the Cross Validation
# Evaluate best model
best_score=evaluator.evaluate(predictions)
# print('Best model score : %03f' % best_score)

## 6. Predicting test data

We are using the previously created pipleline on the corrected dataset: df_test

In [111]:
df_test.groupBy('label').count().show()

+-----+------+
|label| count|
+-----+------+
|    1|250436|
|    0| 60593|
+-----+------+



In [112]:
transformer_test = pipeline.fit(df_test)
transformed_df_test = transformer_test.transform(df_test)

# Keep relevant columns
selection_test = ["label", "features"] #+ assemblerInputs #  "duration", "src_bytes"
dataset_test = transformed_df_test.select(selection_test)

In [113]:
dataset_test.limit(5).toPandas()

,label,features
0,0,"(2.0, 1.0, 0.0, 0.0, 105.0, 146.0, 0.0, 0.0, 0.0)"
1,0,"(2.0, 1.0, 0.0, 0.0, 105.0, 146.0, 0.0, 0.0, 0.0)"
2,0,"(2.0, 1.0, 0.0, 0.0, 105.0, 146.0, 0.0, 0.0, 0.0)"
3,1,"(2.0, 1.0, 0.0, 0.0, 105.0, 146.0, 0.0, 0.0, 0.0)"
4,1,"(2.0, 1.0, 0.0, 0.0, 105.0, 146.0, 0.0, 0.0, 0.0)"


In [114]:
# Use test set here so we can measure the accuracy of our model on new data
predictions_test = cvModel.transform(dataset)        # WE CAN EITHER USE MODEL OR CVMODEL
# cvModel uses the best model found from the Cross Validation

In [115]:
dataset.limit(5).toPandas()

,label,features
0,0,"(1.0, 2.0, 0.0, 0.0, 215.0, 45076.0, 0.0, 0.0,..."
1,0,"(1.0, 2.0, 0.0, 0.0, 162.0, 4528.0, 0.0, 0.0, ..."
2,0,"(1.0, 2.0, 0.0, 0.0, 236.0, 1228.0, 0.0, 0.0, ..."
3,0,"(1.0, 2.0, 0.0, 0.0, 233.0, 2032.0, 0.0, 0.0, ..."
4,0,"(1.0, 2.0, 0.0, 0.0, 239.0, 486.0, 0.0, 0.0, 0.0)"


In [116]:
predictions_test.limit(2).toPandas()

,label,features,rawPrediction,probability,prediction
0,0,"(1.0, 2.0, 0.0, 0.0, 215.0, 45076.0, 0.0, 0.0,...","[2.26946088401, -2.26946088401]","[0.906316022911, 0.0936839770893]",0.0
1,0,"(1.0, 2.0, 0.0, 0.0, 162.0, 4528.0, 0.0, 0.0, ...","[2.27248203687, -2.27248203687]","[0.906572226116, 0.0934277738836]",0.0


In [117]:
dataset_test.limit(5).toPandas()

,label,features
0,0,"(2.0, 1.0, 0.0, 0.0, 105.0, 146.0, 0.0, 0.0, 0.0)"
1,0,"(2.0, 1.0, 0.0, 0.0, 105.0, 146.0, 0.0, 0.0, 0.0)"
2,0,"(2.0, 1.0, 0.0, 0.0, 105.0, 146.0, 0.0, 0.0, 0.0)"
3,1,"(2.0, 1.0, 0.0, 0.0, 105.0, 146.0, 0.0, 0.0, 0.0)"
4,1,"(2.0, 1.0, 0.0, 0.0, 105.0, 146.0, 0.0, 0.0, 0.0)"


In [118]:
# Evaluate best model
best_score_test = evaluator.evaluate(predictions_test)
print('Best model score : %03f' % best_score_test)

Best model score : 0.942685


In [ ]:
# spark.stop()